In [1]:
import numpy
from helper import *
from matplotlib import pyplot
%matplotlib inline

# Steepest Descent

In [2]:
pi = numpy.pi
Lx = 1.
Ly = 1.
nx, ny = 101, 101
x = numpy.linspace( 0., Lx, nx )
y = numpy.linspace( -0.5, .5, ny )
dx = Lx / ( nx - 1 ) 
dy = Ly / ( ny - 1 )
X, Y = numpy.meshgrid( x, y )
b = -2 * pi**2 / 4 * numpy.sin( pi * X / Lx ) * numpy.cos( pi * Y / Ly )

p0 = numpy.zeros((ny, nx))

p_exact = poisson_solution(x, y, Lx, Ly)

In [3]:
def laplacian( p, dx ):

    l = (-4.0 * p[1:-1, 1:-1] + p[1:-1, :-2] + p[1:-1, 2:] + p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    
    return l

In [4]:
def poisson_2d_st(p0, b, dx, dy, maxiter = 20000, rtol = 1e-6 ):
    
    p = p0.copy()
    r = numpy.zeros_like(p)  # initial residual
    Ar = numpy.zeros_like(p)  # to store the mat-vec multiplication
    conv = []  # convergence history
    diff = rtol + 1  # initial difference
    ites = 0  # iteration index
    
    while diff > rtol and ites < maxiter:
        
        pk = p.copy()
        r[ 1:-1, 1:-1 ] = b[ 1:-1, 1:-1 ] - laplacian( pk, dx )
        Ar[ 1:-1, 1:-1 ] = laplacian( r, dx )
        alpha = numpy.sum( r * r ) / numpy.sum( r * Ar )
        p = pk + alpha * r
        
        diff = l2_norm(p, pk)
        conv.append(diff)
        ites += 1
    
    return ites, conv, p

In [5]:
ites, conv, p = poisson_2d_st(p0, b, dx, dy, maxiter = 20000, rtol = 1e-10  )

In [6]:
print('Method of steepest descent: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv[-1]))

Method of steepest descent: 2 iterations to reach a relative difference of 1.3307695446303778e-16


# Conjugate Gradient

In [18]:
def poisson_2d_cg(p0, b, dx, dy, maxiter = 20000, rtol = 1e-6 ):
    
    p = p0.copy()
    r = numpy.zeros_like(p)  # initial residual
    Ad = numpy.zeros_like(p)  # to store the mat-vec multiplication
    r[ 1:-1, 1:-1 ] = b[ 1:-1, 1:-1 ] - laplacian( p, dx )
    d = r.copy()
    conv = []  # convergence history
    diff = rtol + 1  # initial difference
    ites = 0  # iteration index
    
    while diff > rtol and ites < maxiter:
        
        pk = p.copy()
        rk = r.copy()
        
        Ad[ 1:-1, 1:-1 ] = laplacian( d, dx )
        alpha = numpy.sum( r * r ) / numpy.sum( d * Ad )
        p = pk + alpha * d
        r = rk - alpha * Ad
        Beta = numpy.sum( r * r ) / numpy.sum( rk * rk )
        d = r + Beta * d
        
        diff = l2_norm(p, pk)
        conv.append(diff)
        ites += 1
            
    return ites, conv, p

In [19]:
ites, conv, p = poisson_2d_cg( p0, b, dx, dy, maxiter = 20000, rtol = 1e-10 )

In [20]:
ites

2

In [21]:
b2 = numpy.sin( pi * X / Lx ) * numpy.cos( pi * Y / Ly ) + numpy.sin( 6 * pi * X / Lx ) * numpy.cos( 6 * pi * Y / Ly )

In [22]:
ites, conv, p = poisson_2d_cg( p0, b2, dx, dy, maxiter = 20000, rtol = 1e-10 )

In [23]:
ites

72

In [26]:
ites, conv, p = poisson_2d_st( p0, b2, dx, dy, maxiter = 40000, rtol = 1e-10 )

In [27]:
ites

31591

In [28]:
p, ites, conv = poisson_2d_jacobi(p0, b2, dx, dy, maxiter = 40000, rtol = 1e-10)

In [29]:
ites

31226